In [7]:
from PIL import Image, ImageFilter, ImageEnhance

def blend_object_with_background(base_image_path, replacement_image_path, bbox_coords):
    """
    Replaces the object in the base image (bounded by bbox_coords) with a replacement image and blends it.
    
    Args:
    base_image_path (str): Path to the base image (where the object will be replaced).
    replacement_image_path (str): Path to the replacement image.
    bbox_coords (tuple): Bounding box coordinates as (x1, y1, x2, y2).
    
    Returns:
    Image: The image with the object replaced and blended into the background.
    """
    # Load the base image and the replacement image
    base_image = Image.open(base_image_path).convert('RGB')
    replacement_image = Image.open(replacement_image_path).convert('RGBA')  # Ensure it has alpha channel
    
    # Extract bounding box coordinates
    x1, y1, x2, y2 = bbox_coords
    
    # Calculate the size of the bounding box
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # Resize the replacement image to fit inside the bounding box
    replacement_image = replacement_image.resize((bbox_width, bbox_height))
    
    # Create a mask for the replacement image by applying a Gaussian blur for smooth blending
    mask = replacement_image.split()[3]  # Get alpha channel as the mask
    mask = mask.filter(ImageFilter.GaussianBlur(10))  # Apply Gaussian blur to the mask for blending
    
    # Paste the replacement image onto the base image at the bounding box location using the blurred mask
    base_image.paste(replacement_image, (x1, y1), mask)
    
    # Optionally enhance brightness/contrast to blend it better
    enhancer = ImageEnhance.Brightness(base_image)
    base_image = enhancer.enhance(1.05)  # Slightly brighten the image for a better blend

    # Return the modified image
    return base_image

# Example usage
base_image_path = "./demo/source.jpg"  # Replace with your base image path
replacement_image_path = "./demo/lampobj.jpg"  # Replace with your replacement image path
bbox_coords = (1660, 780, 1720, 860)  # Example bounding box coordinates (x1, y1, x2, y2)

# Perform the replacement and blending
result_image = blend_object_with_background(base_image_path, replacement_image_path, bbox_coords)

# Save or show the result
result_image.show()  # Show the image
result_image.save("output_image_blended.png")  # Save the modified image
